#  Chapter 2

### imports


In [1]:
import os
import pathlib


import pandas as pd
import torch

## Data manipulation

### Getting started

In [38]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [39]:
x.shape # dimensions of each axis

torch.Size([12])

In [40]:
x.numel() # product of shape (total elements)

12

In [41]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [42]:
# using -1 to fill in dimension
# x.reshape(6,-1)
x.reshape(-1,3)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

In [43]:
torch.randn(3, 4) # random initialisation

tensor([[-0.3673,  0.7722,  0.4467, -0.2710],
        [ 0.5529,  1.0887, -0.1331,  0.0601],
        [ 2.1426,  1.3112,  1.7003,  0.2752]])

In [44]:
torch.tensor([[[1]],[[7]]]).shape

torch.Size([2, 1, 1])

In [45]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y, torch.exp(x)  # The ** operator is exponentiation

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]),
 tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03]))

In [46]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)
# similar to numpy!

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [49]:
X == Y # familiar!

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [50]:
X < Y # familiar!

tensor([[ True, False,  True, False],
        [False, False, False, False],
        [False, False, False, False]])

In [51]:
X > Y # familiar!

tensor([[False, False, False, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]])

### Broadcasting

In [51]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a +  b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [61]:
a = torch.arange(12).reshape((2, 6))
b = torch.arange(2).reshape((2, 1)) 
a + b

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 7,  8,  9, 10, 11, 12]])

### Indexing

In [64]:
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [65]:
X[-1], X[1:3,1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 5.,  6.],
         [ 9., 10.]]))

In [66]:
X[1, 2] = 999
X

tensor([[  0.,   1.,   2.,   3.],
        [  4.,   5., 999.,   7.],
        [  8.,   9.,  10.,  11.]])

In [67]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

### Saving memory

In [68]:
# after the operation Y points to a different memory location
before = id(Y)
Y = Y + X
id(Y) == before

False

In [72]:
# using [:] means the same memory location !
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 4940822800
id(Z): 4940822800


### to numpy and back

In [1]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

NameError: name 'X' is not defined

## Data pre-processing

In [28]:
def mkdir_if_not_exist(path):
    """Make a directory if it does not exist."""
    if not isinstance(path, str):
        path = os.path.join(*path)
    if not os.path.exists(path):
        os.makedirs(path)

data_file = '../data/house_tiny.csv'
mkdir_if_not_exist('../data')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # Column names
    f.write('NA,Pave,127500\n')  # Each row represents a data example
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [29]:
# load data into pandas

data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [32]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [33]:
# categorical imputation using get dummies (I don't much like this)
inputs = pd.get_dummies(inputs, dummy_na=True) 
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


In [37]:
# Convert to tensor from numpy array
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## Linear algebra

### Scalars

In [5]:
import torch

x = torch.tensor([3.0])
y = torch.tensor([2.0])

x + y, x * y, x / y, x**y

(tensor([5.]), tensor([6.]), tensor([1.5000]), tensor([9.]))

### Vectors

In [9]:
x = torch.arange(4)
x

tensor([0, 1, 2, 3])

In [10]:
print(len(x))
print(x.shape)

4
torch.Size([4])


### Matrices

In [11]:
A = torch.arange(20).reshape(5, 4)
A

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])

In [12]:
A.T

tensor([[ 0,  4,  8, 12, 16],
        [ 1,  5,  9, 13, 17],
        [ 2,  6, 10, 14, 18],
        [ 3,  7, 11, 15, 19]])

### Tensors

In [13]:
X = torch.arange(24).reshape(2, 3, 4)
X

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

In [14]:
X[:,:,-1]

tensor([[ 3,  7, 11],
        [15, 19, 23]])

### Tensor Arithmetic

In [19]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
B = A.clone()  # Assign a copy of `A` to `B` by allocating new memory
# why are they cloning A?
A, A + B

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]),
 tensor([[ 0.,  2.,  4.,  6.],
         [ 8., 10., 12., 14.],
         [16., 18., 20., 22.],
         [24., 26., 28., 30.],
         [32., 34., 36., 38.]]))

In [20]:
# hadmard
A * B

tensor([[  0.,   1.,   4.,   9.],
        [ 16.,  25.,  36.,  49.],
        [ 64.,  81., 100., 121.],
        [144., 169., 196., 225.],
        [256., 289., 324., 361.]])

In [21]:
# scalar multiplication
a = 2
X = torch.arange(24).reshape(2, 3, 4)
a + X, (a * X).shape

(tensor([[[ 2,  3,  4,  5],
          [ 6,  7,  8,  9],
          [10, 11, 12, 13]],
 
         [[14, 15, 16, 17],
          [18, 19, 20, 21],
          [22, 23, 24, 25]]]),
 torch.Size([2, 3, 4]))

### reduction

In [22]:
A.shape, A.sum()

(torch.Size([5, 4]), tensor(190.))

In [24]:
A.sum(axis=1), A.sum(axis=0)

(tensor([ 6., 22., 38., 54., 70.]), tensor([40., 45., 50., 55.]))

In [26]:
# the mean
A.mean(), A.sum() / A.numel()

(tensor(9.5000), tensor(9.5000))

In [30]:
sum_A = A.sum(axis=1, keepdims=True)
sum_A

tensor([[ 6.],
        [22.],
        [38.],
        [54.],
        [70.]])

In [45]:
# The following line errors! You have to keep dims
# A / A.sum(axis=1)

# A.shape
# A.sum(axis=1).shape
# A + A.sum(axis=1)

In [46]:
A / A.sum(axis=1,keepdims=True)

tensor([[0.0000, 0.1667, 0.3333, 0.5000],
        [0.1818, 0.2273, 0.2727, 0.3182],
        [0.2105, 0.2368, 0.2632, 0.2895],
        [0.2222, 0.2407, 0.2593, 0.2778],
        [0.2286, 0.2429, 0.2571, 0.2714]])

In [47]:
A.cumsum(axis=0)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  6.,  8., 10.],
        [12., 15., 18., 21.],
        [24., 28., 32., 36.],
        [40., 45., 50., 55.]])

### Dot Products

In [53]:
x = torch.arange(4, dtype=torch.float32)
y = torch.ones(4, dtype = torch.float32)
x, y, torch.dot(x, y)
# x @ y # equivalent
# torch.sum(x * y) # equivalent

(tensor([0., 1., 2., 3.]), tensor([1., 1., 1., 1.]), tensor(6.))

In [58]:
A.shape, x.shape, torch.mv(A, x)
# A @ x

(torch.Size([5, 4]), torch.Size([4]), tensor([ 14.,  38.,  62.,  86., 110.]))

In [70]:
B = torch.ones(4, 3)
torch.mm(A, B)
# A @ B

tensor([[ 6.,  6.,  6.],
        [22., 22., 22.],
        [38., 38., 38.],
        [54., 54., 54.],
        [70., 70., 70.]])

### Norms

In [76]:
u = torch.tensor([3.0, -4.0])
torch.norm(u, p=2)

tensor(5.)